In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install mediapipe

In [ ]:
import cv2
import librosa
import numpy as np
import pandas as pd
import sklearn 
import os
import matplotlib.pyplot as plt
import mediapipe as mp
from torchvision import transforms
import shutil
from PIL import Image
from google.colab.patches import cv2_imshow
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torch.optim as optim
import time
import copy
import os
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix, classification_report
import seaborn as sns



In [ ]:
path = "/content/drive/MyDrive/4010/Image/A"
o_path = "/content/drive/MyDrive/4010/skeleton_image/A"
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
for i in range(2,9):
  data_path = path+str(i)+"/"
  output_path = o_path+str(i)+"/"
  print(data_path)
  for filename in os.listdir(data_path):
    image_path = data_path+filename
    text = ""
    with mp_pose.Pose(
      # these two values are default setting already work pretty well
      # higher confidence = higher accuracy of the detection model
        model_complexity = 2,
        min_detection_confidence=0.4,  
        min_tracking_confidence=0.5) as pose:

        image = cv2.imread(image_path, -1)

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)    
        
        image.flags.writeable = False

        # make detection
        results = pose.process(image)


        # Draw the pose annotation on the image.
        image.flags.writeable = True
      
        
        # recolor the image to BGR format
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        img = cv2.imread('./black.jpg')
        annotated_image = img.copy()
        mp_drawing.draw_landmarks(
        img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0,255,255), thickness=5, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0,255,255), thickness=5, circle_radius=4))
        
        # cv2_imshow(img)
        
        p = transforms.Compose([transforms.Resize((100,100))])
        hi = Image.fromarray(img)
        img = p(hi)
        oput = np.array(img)

        cv2.imwrite(os.path.join(output_path)+ filename[:-4]+'.jpg', oput)


# Framing images

In [ ]:
def framing(path):
    # Load 10 images
    images = []
    for filename in os.listdir(path):
      img_path = path+filename
      img = cv2.imread(img_path)
      images.append(img)

      if len(images)==10:
      # Concatenate images horizontally
        result = np.hstack(images)
        # Display result
        cv2_imshow(result)
        break
      
framing("/content/drive/MyDrive/4010/skeleton_image/A2/")

# Coordinate vector extracting

In [ ]:
data = {"labels":[], "feature":[]}
path = "/content/drive/MyDrive/4010/Image/"
mp_pose = mp.solutions.pose

for label_index, filename in enumerate(os.listdir(path)):
  class_dir = path+"/"+filename
  ct = 0
  for file in os.listdir(class_dir):
    file_path = class_dir+"/"+file
    text=""
    with mp_pose.Pose(
    # these two values are default setting already work pretty well
    # higher confidence = higher accuracy of the detection model
      model_complexity = 2,
      min_detection_confidence=0.5,  
      min_tracking_confidence=0.5) as pose:

      image = cv2.imread(file_path, -1)
      image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)    
      
      image.flags.writeable = False

      # make detection
      results = pose.process(image)

      
      if results.pose_landmarks:
        for idx,pose_result in enumerate(results.pose_landmarks.landmark):
          # 11-32 is the nodes of body except the face
          if( idx >=11 and idx<=32):
            text = text + str(pose_result.x) + " " + str(pose_result.y) + " "

      # ct +=1
      # if ct ==10:
      data["labels"].append(filename)
      data["feature"].append(text)



df = pd.DataFrame(data, columns = ["labels","feature"])
df.to_csv('/content/data.csv', index=False)

# Resnet for skeleton image input

In [ ]:
import warnings
warnings.filterwarnings("ignore")
 
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=(1, 1), stride=(2, 2), bias=False),
                nn.BatchNorm2d(out_channels,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential()

        self.bn1 = nn.BatchNorm2d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            
    def forward(self, input):
        shortcut = self.shortcut(input)
        input = nn.ReLU()(self.bn1(self.conv1(input)))
        input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = input + shortcut
        return nn.ReLU()(input)


class ResNet(nn.Module):
    def __init__(self, resblock, outputs=10):
        super().__init__()
        self.layer0 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.layer1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.layer2 = nn.ReLU(inplace=True)
        self.layer3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        self.layer4 = nn.Sequential(resblock(64, 64, downsample=False))
        self.layer5 = nn.Sequential(resblock(64, 128, downsample=True))
        self.layer6 = nn.Sequential(resblock(128, 256, downsample=True))
        self.layer7 = nn.Sequential(resblock(256, 512, downsample=True))

        self.fc = nn.Linear(65536, outputs)

    def forward(self, input):
        input = self.layer0(input)
        input = self.layer1(input)
        input = self.layer2(input)
        input = self.layer3(input)
        input = self.layer4(input)
        input = self.layer5(input)
        input = self.layer6(input)
        input = self.layer7(input)
        input = torch.flatten(input, start_dim=1)
        input = self.fc(input)
        probs = F.softmax(input, dim=1)


        return input,probs

In [ ]:

class ImageFlowDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform):
        self.root_dir = root_dir
        self.transform = transform
        self.class_names = os.listdir(root_dir)
        self.classes = sorted(self.class_names)
        self.class_to_idx = {name: i for i, name in enumerate(self.class_names)}
        self.image_filenames = []
        self.labels = []
        for class_name in self.class_names:
            class_dir = os.path.join(root_dir, class_name)
            for i, image_name in enumerate(os.listdir(class_dir)):
                image_path = os.path.join(class_dir, image_name)
                self.image_filenames.append(image_path)
                self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        images = []
        for i in range(10):
            image_idx = idx+i
            if image_idx < len(self.image_filenames):
                image_path = self.image_filenames[image_idx]
                image = cv2.imread(image_path)
                images.append(image)
            else:
                # if index is out of range, return an empty image
                empty_image = np.zeros((100, 100, 3), dtype=np.uint8)
                images.append(empty_image)
        image_flow = np.hstack(images)
        image_flow = self.transform(image_flow)
        label = self.labels[idx]
        return image_flow, torch.tensor(label)

# Data augmentation
data_transforms =transforms.Compose([
                  transforms.ToTensor(),
                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                  transforms.RandomApply([transforms.Lambda(lambda x: x + torch.randn_like(x)*0.1)], p=0.5)
        ])

data_dir = '/content/drive/MyDrive/4010/skeleton_image'

res_dataset = ImageFlowDataset(root_dir=data_dir, transform=data_transforms)

res_train_length = int(len(res_dataset)*0.8)
res_test_length = int(len(res_dataset))-res_train_length

res_train_dataset,res_test_dataset=torch.utils.data.random_split(res_dataset,[res_train_length,res_test_length])

res_train_loader = DataLoader(res_train_dataset, batch_size=16, shuffle=False, drop_last=True)
res_test_loader = DataLoader(res_test_dataset, batch_size=16, shuffle=False, drop_last=True)
class_names = res_dataset.classes
print(class_names)
if torch.cuda.is_available(): 
     device = "cuda" 
     print("using cuda")
else: 
     device = "cpu"

In [ ]:
def train_model(model, criterion,optimizer,train_loader, num_epochs=10):
    model.train() 
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # Forward
        outputs, probs = model(inputs)
        _, preds = torch.max(probs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        # Backward
        loss.backward()
        optimizer.step()
        total += labels.size(0)
        correct += (preds == labels).sum().item()           

    epoch_acc = correct / total
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print()      
    return model, optimizer, epoch_loss

In [ ]:
def val_model(model, criterion, optimizer, test_loader, num_epochs=10):
    model.eval() 
    running_loss = 0.0
    correct = 0
    total = 0
    y_pred=[]
    y_true=[]
    for inputs, labels in test_loader:   
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # Forward
        outputs,probs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(probs, 1)

        running_loss += loss.item() * inputs.size(0)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

        pred = outputs.data.argmax(1).cpu().numpy()
        y_pred.extend(pred)
        true = labels.data.cpu().numpy()
        y_true.extend(true)

    epoch_acc = correct / total
    epoch_loss = running_loss / len(test_loader.dataset)
    print(f'Testing Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print()      
    return model, epoch_loss, epoch_acc, y_pred, y_true

In [ ]:
def training_loop(model, criterion, optimizer, epochs, train_dataloader, test_dataloader):
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  # set objects for storing metrics
  train_loss = []
  val_loss = []
  y_trues=[]
  y_preds=[]
  # Train model
  for epoch in range(0, epochs):
    print(f'Epoch: {epoch+1}\t')
    # training
    model, optimizer, t_loss = train_model(model, criterion, optimizer, train_dataloader,num_epochs=epochs)
    train_loss.append(train_loss)

    # # validation
    with torch.no_grad():
      model, v_loss, acc, y_pred, y_true = val_model(model, criterion, optimizer, test_dataloader,num_epochs=epochs)
      val_loss.append(v_loss)
      y_trues.extend(y_true)
      y_preds.extend(y_pred)
    if acc > best_acc:
      best_acc = acc
      best_model_wts = copy.deepcopy(model.state_dict())

  print(f'Best val Acc: {best_acc:4f}')
  model.load_state_dict(best_model_wts)

  cf_matrix = confusion_matrix(y_true, y_pred)
  # Create pandas dataframe
  dataframe = pd.DataFrame(cf_matrix, index=class_names, columns=class_names)
  sns.heatmap(dataframe, annot=True, cbar=None,cmap="YlGnBu",fmt="d")
  plt.title("Confusion Matrix"), plt.tight_layout()
  plt.ylabel("True Class"), 
  plt.xlabel("Predicted Class")
  plt.show()

  return train_loss,val_loss

In [ ]:
if __name__ == '__main__':            
  model = ResNet(Block).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  criterion = nn.CrossEntropyLoss()
  t_loss, v_loss = training_loop(model, criterion, optimizer,100, res_train_loader, res_test_loader) 

# Using CNN for accelerometer data

In [ ]:
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.max_pool = nn.AdaptiveMaxPool1d(1)
        self.fc1   = nn.Conv1d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Conv1d(in_planes // ratio, in_planes, 1, bias=False)  
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        out = self.sigmoid(out)
        return out

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        
        self.conv1 = nn.Conv1d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        x = self.sigmoid(x)
        return x

class CBAMBlock(nn.Module):
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super(CBAMBlock, self).__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(kernel_size)
        
    def forward(self, x):
        out = x * self.ca(x)
        out = out * self.sa(out)
        return out

class CBAMModel(nn.Module):
    def __init__(self,outputs=8):
        super(CBAMModel,self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=14, out_channels=64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.cbam1 = CBAMBlock(64)
        self.relu = nn.ReLU(inplace=True)

        self.pool1 = nn.MaxPool1d(2)
        
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(128)
        self.cbam2 = CBAMBlock(128)
        self.pool2 = nn.MaxPool1d(2)
        self.f = nn.Flatten()
        self.fc = nn.Linear(128, 64)
        self.output= nn.Linear(64,outputs)

    def forward(self, input):
        input = self.conv1(input)
        input = self.bn1(input)
        input = self.relu(input)
        input = self.cbam1(input)
        input = self.pool1(input)

        input = self.conv2(input)
        input = self.bn2(input)
        input = self.relu(input)
        input = self.cbam2(input)

        input = self.f(input)
        input = self.fc(input)
        input = self.output(input)
        probs = F.softmax(input, dim=1)

        return input,probs

In [ ]:
import torch
from torch.utils.data import Dataset
import os
import numpy as np

class AccelDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.classes = os.listdir(root_dir)

    def __len__(self):
        total_data = 0
        for c in self.classes:
            data_dir = os.path.join(self.root_dir, c)
            total_data += len(os.listdir(data_dir))
        return total_data

    def __getitem__(self, idx):
        class_idx = 0
        while True:
            data_dir = os.path.join(self.root_dir, self.classes[class_idx])
            num_data = len(os.listdir(data_dir))
            if idx < num_data:
                break
            else:
                idx -= num_data
                class_idx += 1
        file_path = os.path.join(data_dir, os.listdir(data_dir)[idx])

        data = np.loadtxt(file_path)[:, :-1] # remove the last column
        
        # Padding to ensure that all data has the same length
        max_length = 14
        current_length = data.shape[0]
        if current_length < max_length:
            padding = np.zeros((max_length-current_length, 18))
            data = np.concatenate((data, padding), axis=0)
        elif current_length > max_length:
            data = data[:max_length, :]

        label = class_idx
        return torch.tensor(data).float(), torch.tensor(label)


data_dir = '/content/drive/MyDrive/4010/accel'

CBAM_dataset = AccelDataset(root_dir=data_dir)

CBAM_train_length = int(len(CBAM_dataset)*0.8)
CBAM_test_length = int(len(CBAM_dataset))-CBAM_train_length

CBAM_train_dataset,CBAM_test_dataset=torch.utils.data.random_split(CBAM_dataset,[CBAM_train_length,CBAM_test_length])

CBAM_train_loader = DataLoader(CBAM_train_dataset, batch_size=16, shuffle=False, drop_last=True)
CBAM_test_loader = DataLoader(CBAM_test_dataset, batch_size=16, shuffle=False, drop_last=True)
class_names = CBAM_dataset.classes
print(class_names)
if torch.cuda.is_available(): 
     device = "cuda" 
     print("using cuda")
else: 
     device = "cpu"

In [ ]:
if __name__ == '__main__':            
  model = CBAMModel(outputs=8).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
  criterion = nn.CrossEntropyLoss()
  t_loss, v_loss = training_loop(model, criterion, optimizer,100, CBAM_train_loader, CBAM_train_loader) 

In [ ]:
import matplotlib.pyplot as plt
 
plt.title("Validation loss of model")
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.plot(v_loss, label='Validation Loss')
plt.show()

# Using BILSTM for coordinate data

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, bidirectional=True, num_layers=num_layers)
        self.fc = nn.Linear(1024, num_classes)
        self.f = nn.Flatten()
        self.fc1 = nn.Linear(440, input_size)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(2 * 2, x.size(0), 512).to(device)
        c0 = torch.zeros(2 * 2, x.size(0), 512).to(device)
        
        x = self.f(x)
        x = self.fc1(x)
        # Forward propagate LSTM
        out, _ = self.lstm(x)
        
        
        # Decode the hidden state of the last time step
        out = self.fc(out)
        probs = F.softmax(out, dim=1)

        
        return out,probs

In [ ]:
label_map = {'A1': 0, 'A2': 1, 'A3': 2, 'A4': 3, 'A5': 4, 'A6': 5, 'A7': 6, 'A8': 7}

class CoordinateDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path):
        self.data = pd.read_csv(csv_path,skiprows=1)
        self.classes = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8']
        self.data.fillna('0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0', inplace=True)  # replace missing values with 0

    def __getitem__(self, idx):
        # Select 10 consecutive rows from the CSV file
        rows = self.data.iloc[idx:idx+10]
        # Merge the 10 rows into a single input vector
        features = []
        for feature in rows.iloc[:, 1]:
          tmp = np.fromstring(feature, dtype=float, sep=' ')
          features.append(tmp)
        
        features = np.array(features)
        input_vector = torch.from_numpy(features)
        input_vector = input_vector.to(torch.float32)
        # Extract the label from the first row and convert it to an integer
        label = label_map[str(rows.iloc[0, 0])]
        return input_vector, torch.tensor(label)
    
    def __len__(self):
        return len(self.data) - 9

  
# Data augmentation
csv_dir = '/content/data.csv'

BiLSTM_dataset = CoordinateDataset(csv_path = csv_dir)

BiLSTM_train_length = int(len(BiLSTM_dataset)*0.8)
BiLSTM_test_length = int(len(BiLSTM_dataset))-BiLSTM_train_length

BiLSTM_train_dataset,BiLSTM_test_dataset=torch.utils.data.random_split(BiLSTM_dataset,[BiLSTM_train_length,BiLSTM_test_length])

BiLSTM_train_loader = DataLoader(BiLSTM_train_dataset, batch_size=16, shuffle=False, drop_last=True)
BiLSTM_test_loader = DataLoader(BiLSTM_test_dataset, batch_size=16, shuffle=False, drop_last=True)
class_names = BiLSTM_dataset.classes
print(class_names)
if torch.cuda.is_available(): 
     device = "cuda" 
     print("using cuda")
else: 
     device = "cpu"

In [ ]:
if __name__ == '__main__':            
  model = BiLSTM(input_size=75,hidden_size=512,num_layers=2,num_classes=8).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  criterion = nn.CrossEntropyLoss()
  t_loss, v_loss = training_loop(model, criterion, optimizer,100,BiLSTM_train_loader,BiLSTM_test_loader) 

# Fusion Model

In [ ]:
class Fusion_Three(nn.Module):
    def __init__(self):
        super(Fusion_Three, self).__init__()
        self.BILSTM = BiLSTM(input_size=75,hidden_size=512,num_layers=2,num_classes=12800)
        self.resnet = ResNet(Block,outputs = 512)
        self.CBAM = CBAMModel(outputs=256)      
        self.drop = nn.Dropout()
        self.fc1 = nn.Linear(13568, 768)
        self.fc2 = nn.Linear(768,512)
        self.fc_out = nn.Linear(512, 8)
        
    def forward(self, x1, x2, x3):
        
        x1,_ = self.BILSTM(x1)
        x1 = x1.view(x1.size(0), -1)
        
        x2,_ = self.resnet(x2)
        x2 = x2.view(x2.size(0), -1)
        
        x3,_ = self.CBAM(x3)
        x3 = x3.view(x3.size(0), -1)

        # Concatenate in dim1 (feature dimension)
        x = torch.cat((x1, x2, x3), 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc_out(x)
        probs = F.softmax(x, dim=1)

        return x,probs


class Fusion_Two(nn.Module):
    def __init__(self):
        super(Fusion_Two, self).__init__()
        self.resnet = ResNet(Block,outputs = 512)
        self.CBAM = CBAMModel(outputs=256)      
        self.fc1 = nn.Linear(768, 768)
        self.fc2 = nn.Linear(768,512)
        self.fc_out = nn.Linear(512, 8)
        
    def forward(self, x1, x2):
        
        x1,_ = self.CBAM(x1)
        x1 = x1.view(x1.size(0), -1)
        
        x2,_ = self.resnet(x2)
        x2 = x2.view(x2.size(0), -1)

        # Concatenate in dim1 (feature dimension)
        x = torch.cat((x1, x2), 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc_out(x)
        probs = F.softmax(x, dim=1)

        return x,probs

In [ ]:
def train_fusion_model_3(model, criterion,optimizer,num_epochs=10):
    model.train() 
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_BiLSTM, batch_res, batch_CBAM in zip(BiLSTM_train_loader,res_train_loader, CBAM_train_loader):
        inputs_BiLSTM, labels = batch_BiLSTM
        inputs_res, labels = batch_res
        inputs_CBAM, labels = batch_CBAM

        inputs_BiLSTM = inputs_BiLSTM.to(device)
        inputs_res = inputs_res.to(device)
        inputs_CBAM = inputs_CBAM.to(device)
        labels = labels.to(device)


        optimizer.zero_grad()
        # Forward
        outputs, probs = model(inputs_BiLSTM,inputs_res,inputs_CBAM)
        _, preds = torch.max(probs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * (inputs_BiLSTM.size(0)+inputs_CBAM.size(0)+inputs_res.size(0))
        # Backward
        loss.backward()
        optimizer.step()
        total += labels.size(0)
        correct += (preds == labels).sum().item()
    epoch_acc = correct / total
    epoch_loss = running_loss / (len(BiLSTM_train_loader.dataset)+len(CBAM_train_loader.dataset)+len(res_train_loader.dataset))
    print(f'Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print()      
    return model, optimizer, epoch_loss

In [ ]:
def train_fusion_model_2(model, criterion,optimizer,num_epochs=10):
    model.train() 
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_res, batch_CBAM in zip(res_train_loader, CBAM_train_loader):
        inputs_res, labels = batch_res
        inputs_CBAM, labels = batch_CBAM

        inputs_res = inputs_res.to(device)
        inputs_CBAM = inputs_CBAM.to(device)
        labels = labels.to(device)


        optimizer.zero_grad()
        # Forward
        outputs, probs = model(inputs_CBAM,inputs_res)
        _, preds = torch.max(probs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * (inputs_CBAM.size(0)+inputs_res.size(0))
        # Backward
        loss.backward()
        optimizer.step()
        total += labels.size(0)
        correct += (preds == labels).sum().item()
    epoch_acc = correct / total
    epoch_loss = running_loss / (len(CBAM_train_loader.dataset)+len(res_train_loader.dataset))
    print(f'Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print()      
    return model, optimizer, epoch_loss

In [ ]:
def val_fusion_model_3(model, criterion, optimizer, num_epochs=10):
    model.eval() 
    running_loss = 0.0
    correct = 0
    total = 0
    y_true=[]
    y_pred=[]
    dataloaders= zip(BiLSTM_train_loader,res_train_loader, CBAM_train_loader)
    for batch_BiLSTM, batch_res, batch_CBAM in dataloaders:
        inputs_BiLSTM, labels = batch_BiLSTM
        inputs_res, labels = batch_res
        inputs_CBAM, labels = batch_CBAM

        inputs_BiLSTM = inputs_BiLSTM.to(device)
        inputs_res = inputs_res.to(device)
        inputs_CBAM = inputs_CBAM.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        # Forward
        outputs,probs = model(inputs_BiLSTM,inputs_res,inputs_CBAM)
        loss = criterion(outputs, labels)
        _, preds = torch.max(probs, 1)

        running_loss += loss.item() * (inputs_BiLSTM.size(0)+inputs_CBAM.size(0)+inputs_res.size(0))
        total += labels.size(0)
        correct += (preds == labels).sum().item()

        pred = outputs.data.argmax(1).cpu().numpy()
        y_pred.extend(pred)
        true = labels.data.cpu().numpy()
        y_true.extend(true)

    epoch_acc = correct / total
    epoch_loss = running_loss / (len(BiLSTM_train_loader.dataset)+len(CBAM_train_loader.dataset)+len(res_train_loader.dataset))
    print(f'Testing Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print()      
    return model, epoch_loss, epoch_acc

In [ ]:
def val_fusion_model_2(model, criterion, optimizer, num_epochs=10):
    model.eval() 
    running_loss = 0.0
    correct = 0
    total = 0
    y_true=[]
    y_pred=[]
    dataloaders= zip(res_train_loader, CBAM_train_loader)
    for batch_res, batch_CBAM in dataloaders:
        inputs_res, labels = batch_res
        inputs_CBAM, labels = batch_CBAM

        inputs_res = inputs_res.to(device)
        inputs_CBAM = inputs_CBAM.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        # Forward
        outputs,probs = model(inputs_CBAM,inputs_res)
        loss = criterion(outputs, labels)
        _, preds = torch.max(probs, 1)

        running_loss += loss.item() * (inputs_CBAM.size(0)+inputs_res.size(0))
        total += labels.size(0)
        correct += (preds == labels).sum().item()

        pred = outputs.data.argmax(1).cpu().numpy()
        y_pred.extend(pred)
        true = labels.data.cpu().numpy()
        y_true.extend(true)

    epoch_acc = correct / total
    epoch_loss = running_loss / (len(CBAM_train_loader.dataset)+len(res_train_loader.dataset))
    print(f'Testing Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print()      
    return model, epoch_loss, epoch_acc, y_true, y_pred

In [ ]:
def training_fusion_loop_3(model, criterion, optimizer, epochs):
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  # set objects for storing metrics
  train_loss = []
  val_loss = []
  y_trues=[]
  y_preds=[]
  # Train model
  for epoch in range(0, epochs):
    print(f'Epoch: {epoch+1}\t')
    # training
    model, optimizer, t_loss = train_fusion_model_3(model, criterion, optimizer, num_epochs=epochs)
    train_loss.append(train_loss)

    # # validation
    with torch.no_grad():
      model, v_loss, acc, y_true, y_pred = val_fusion_model_3(model, criterion, optimizer,  num_epochs=epochs)
      val_loss.append(v_loss)
      y_trues.extend(y_true)
      y_preds.extend(y_pred)

    if acc > best_acc:
      best_acc = acc
      best_model_wts = copy.deepcopy(model.state_dict())

  print(f'Best val Acc: {best_acc:4f}')
  model.load_state_dict(best_model_wts)

  cf_matrix = confusion_matrix(y_true, y_pred)
  # Create pandas dataframe
  dataframe = pd.DataFrame(cf_matrix, index=class_names, columns=class_names)
  sns.heatmap(dataframe, annot=True, cbar=None,cmap="YlGnBu",fmt="d")
  plt.title("Confusion Matrix"), plt.tight_layout()
  plt.ylabel("True Class"), 
  plt.xlabel("Predicted Class")
  plt.show()
  return train_loss,val_loss

In [ ]:
def training_fusion_loop_2(model, criterion, optimizer, epochs):
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  # set objects for storing metrics
  train_loss = []
  val_loss = []
  y_trues=[]
  y_preds=[]
  # Train model
  for epoch in range(0, epochs):
    print(f'Epoch: {epoch+1}\t')
    # training
    model, optimizer, t_loss = train_fusion_model_2(model, criterion, optimizer, num_epochs=epochs)
    train_loss.append(train_loss)

    # # validation
    with torch.no_grad():
      model, v_loss, acc, y_true, y_pred = val_fusion_model_2(model, criterion, optimizer,  num_epochs=epochs)
      val_loss.append(v_loss)
      y_trues.extend(y_true)
      y_preds.extend(y_pred)

    if acc > best_acc:
      best_acc = acc
      best_model_wts = copy.deepcopy(model.state_dict())

  print(f'Best val Acc: {best_acc:4f}')
  model.load_state_dict(best_model_wts)

  cf_matrix = confusion_matrix(y_true, y_pred)
  # Create pandas dataframe
  dataframe = pd.DataFrame(cf_matrix, index=class_names, columns=class_names)
  sns.heatmap(dataframe, annot=True, cbar=None,cmap="YlGnBu",fmt="d")
  plt.title("Confusion Matrix"), plt.tight_layout()
  plt.ylabel("True Class"), 
  plt.xlabel("Predicted Class")
  plt.show()
  return train_loss,val_loss

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
if __name__ == '__main__':            
  model = Fusion_Two().to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  criterion = nn.CrossEntropyLoss()
  t_loss, v_loss = training_fusion_loop_2(model, criterion, optimizer,100) 

In [ ]:
if __name__ == '__main__':            
  model = Fusion_Three().to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  criterion = nn.CrossEntropyLoss()
  t_loss, v_loss = training_fusion_loop_3(model, criterion, optimizer,100) 